# 직선 검출과 원 검출

## 허프 변환 직선 검출 (hough transform)
![](xyab.png)
> (a)에서 빨간점 (x0, y0)을 지나는 모든 직선(절편들인 a,b는 바뀜) ==> (b)의 빨간 직선!  
  
![](accum.png)  
* xy 공간에서 에지로 판별된 모든 점을 이용하여 ab공간에 직선을 표현  
> **직선이 많이 교차되는 좌표**를 모두 찾아야 함!  
> * 이 때, **축적 배열** 사용 (**ab공간에서** 직선이 지나가는 부분마다 배열값 1씩 증가)
> * 축적 배열(ab공간)에서의 최대값의 좌표 (a, b) ==> 검출된 직선의 매개변수!  

* 실제 허프 변환 구현 시, **극좌표계 형태**의 직선의 방정식 사용!  
    * for 무한대의 기울기 a값을 표현할 수 없기 때문
    * 축적 배열에서 국지적 최대값이 발생하는 위치에서의 rho(p)와 theta 찾기! ==> 직선
    * x, y 공간 -> row, theta 공간 변환
    ![](cossin.png)  
    ![](p.png)

## (실습) 허프 변환 직선 검출
* **cv2.Canny(src, Tlow, Thigh)** : 에지 영상 구하기 (미분 영상) ==> HoughLines()의 입력
* **cv2.HoughLines(src, rho, theta, threshold)** : 직선 파라미터 정보(rho, theta)를 반환하여 빨간 직선 그리기
> rho와 theta 값 : **해상도! (얼마나 잘게 나눠서 볼 것인지)**  
> threshold 값 : **축적 배열에서** ***직선으로 판단할 임계값***

In [1]:
import cv2
import numpy as np
import math

def hough_lines():
    src = cv2.imread('building.png', cv2.IMREAD_GRAYSCALE)
    
    edge = cv2.Canny(src, 50, 150) # 에지들 검출
    lines = cv2.HoughLines(edge, 1, math.pi / 180, 114) # (src, rho, theta, thres)
    
    dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)
    
    if lines is not None:
        for i in range(lines.shape[0]): # line의 개수 (lines의 요소 개수)
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            cos_t = math.cos(theta)
            sin_t = math.sin(theta)
            x0, y0 = rho * cos_t, rho * sin_t
            alpha = 1000 # 이미지 바깥쪽에 위치한 직선 위 점들을 찍기 위한 큰 값
            pt1 = (int(x0 - alpha * sin_t), int(y0 + alpha * cos_t)) #바깥쪽에 있는
            pt2 = (int(x0 + alpha * sin_t), int(y0 - alpha * cos_t)) #직선위두점찍기
            cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)
            
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
        
hough_lines()

#### HoughLines의 T = 250과 T=114 비교 (T를 작게 설정할 수록 더 많은 직선 검출)
> cf. Canny에서도 T low와 T high 값을 조정할 수 있다!

![](hough_lines_T250.png)
![](hough_lines_T114.png)

## 확률적 허프 변환에 의한 직선 검출 (HoughLinesP)
HoughLines()와 같이 직선의 방정식 파라미터 p(rho)와 theta를 반환하는 것이 아니라, 직선의 **시작점과 끝점 좌표**를 반환
> 즉, **선분을 찾는 방법!**    
  
* **cv2.HoughLinesP(src, rho, theta, threshold, minLineLength, maxLineGap)**  ==> return : (x1, y1, x2, y2) 쌍들  
    * minLineLength : 검출할 선분의 최소 길이
    * maxLineGap : 직선으로 간주할 최대 에지 점 간격

In [9]:
def hough_line_segments():
    src = cv2.imread('building.png', cv2.IMREAD_GRAYSCALE)

    edge = cv2.Canny(src, 50, 150)
    lines = cv2.HoughLinesP(edge, 1, math.pi/180, 114, minLineLength=50, maxLineGap=100)
    
    dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)
    print(lines)
    
    if lines is not None:
        print('yes!')
        for i in range(lines.shape[0]):
            pt1 = (lines[i][0][0], lines[i][0][1]) # 시작점과 끝점 좌표 바로 나옴
            pt2 = (lines[i][0][2], lines[i][0][3])
            cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
hough_line_segments()

[[[  4 157 323  59]]

 [[  2 143 323   7]]

 [[  2 128 240   1]]

 [[ 10 172 323 111]]

 [[ 16 173 322 114]]

 [[  0 181 321 170]]

 [[  0 115 178   0]]

 [[ 12 199 281 203]]

 [[129   1 133 203]]]
yes!


## 허프 변환 원 검출
* 중심 좌표가 (a, b)이고 반지름이 r인 원의 방정식 ==> 매개변수 3개 ==> 따라서 허프 변환 그대로 적용 시, **3차원 파라미터 공간에서 축적 배열 정의해야 함**
> 많은 메모리와 연산 시간 필요..  
* 따라서, **허프 그래디언트 방법(Hough gradient method)을 사용하여 원 검출!**
> 그래디언트 이용해 밝기 변화 큰 부분을 원 테두리로 검출  
* **원의 중심 좌표**를 찾는 과정에서 **축적 배열** 사용됨 (단, **xy공간**에서 그대로!)  
![](circle.png)
* 모든 에지의 그래디언트 방향을 따르는 직선 상의 축적 배열 값을 1씩 증가  
> **원의 중심**에서 누적 값 최대!  
* 원의 중심 찾은 후, **다양한 반지름의 원에 대해 원주상에 많은 에지 픽셀이 존재하는지** 확인 후 적절한 **반지름** 선택

* **cv2.HoughCircles(src_gray, cv2.HOUGH_GRADIENT, dp, minDist, param1, param2)**
    * dp : 입력 영상과 축적 배열의 크기 비율 (보통 1)
    * minDist : 인접한 원 중심의 최소 거리 (50)
    * param1 : Canny 에지 검출기의 높은 임계값 (T high)
    * param2 : 축적 배열에서 원 검출을 위한 임계값 (threshold)  
     
> 여기서 입력 영상은 edge 영상이 아닌, **원본 그레이스케일 영상!**  
(에지 영상에는 어느 픽셀이 에지인지만 있고, *그래디언트 정보는 없기 때문*)

## (실습) 허프 변환 원 검출
* 동전 영상에서 원 검출 후, 빨간색으로 표시
* for 잡음 제거, blur() 함수 사용

In [10]:
def hough_circles():
    src = cv2.imread('coins.png', cv2.IMREAD_GRAYSCALE)
    
    blurred = cv2.blur(src, (3, 3)) # 잡음 제거
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 50,
                               param1 = 150, param2 = 30) # 원 검출
    
    dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
    
    if circles is not None:
        print(circles) # 검출된 원 정보들
        for i in range(circles.shape[1]):
            cx, cy, radius = circles[0][i]
            cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)
            
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
hough_circles()

[[[121.5  93.5  26.6]
  [164.5  37.5  30.4]
  [192.5 131.5  32.7]
  [ 40.5 119.5  32.7]
  [261.5 193.5  27.1]
  [238.5  79.5  31.3]
  [130.5 231.5  31.6]
  [186.5 182.5  65.7]
  [104.5 157.5  36.4]
  [ 69.5  58.5  18.8]]]


![](hough_circles.png)